In [5]:
import numpy as np
import matplotlib.pyplot as plt
import kron_pca

In [4]:
ps = 4
pt = 4
cov = np.eye(ps*pt)
X = np.random.multivariate_normal(mean=np.zeros(ps*pt), cov=cov, size=200)
X -= X.mean(axis=0)
sample_cov = np.dot(X.T, X)/len(X)

lambda_theta = 0.2
lambda_gamma = 10000000
num_iter = 100
tau = 0.1

cov_est = kron_pca.prox_grad_robust_toeplitz_kron_pca(sample_cov, ps, pt, lambda_theta, lambda_gamma, num_iter, tau)


plt.imshow(sample_cov)
plt.colorbar()
plt.show()

plt.imshow(cov_est, vmin=-0.1, vmax=1)
plt.colorbar()
plt.show()


NameError: name 'lambda_gamma' is not defined